In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
import keras
import scipy as sp
import scipy.signal
import tensorflow as tf
import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optimizers
import scipy.io as sio
import numpy as np
import sklearn.metrics as metrics 
import matplotlib.pyplot as plt
import statistics

from tensorflow_addons.metrics import RSquare
from tensorflow.keras.models import Sequential
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from wandb.keras import WandbCallback
import wandb
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

2023-04-03 06:04:09.998186: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 06:04:11.367224: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/home/malfonzo/miniconda3/lib/
2023-04-03 06:04:11.367314: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/home/malfonzo/miniconda3/lib/
2023-04-03 06:

In [2]:
F1_test=8
D_test=2
F2_test=16
Drop_test=0 #originally 0.5
KernLength_test=64 #Originally 64 
batch_test=16

N_chan=31
N_samples_long=250
# N_samples_short=251
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-03 06:04:14.223156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-03 06:04:14.234132: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-03 06:04:14.234434: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
# PER SUB 
# ########SUBJECTS=[1,2,3,4,5,6,7,8,9,11,12,13,14,16,17]
SUBJECTS=[9]
for temp_sub in SUBJECTS:
    CLASSES= [50]
    #questa va applicata a for loop di subject che deve essere il più esterno 
    sub="{:02d}".format(temp_sub)

    dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_sub/output", "sub_"+str(sub))
#     os.mkdir(dir00)
    for n_classes in CLASSES:
        #Define loss function
        loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
        LABELS= list(range(0,n_classes))
        
        if n_classes==50:
            numero_classi=5
        else:
            numero_classi=n_classes
        
        evaluation=[]
        iteration=[]
        confusion_matrix_x_test=[]
        confusion_matrix_y_test= []
        validation_acc=[]
        PERFORMANCE=[]

        print("SUBJECT: "+ str(sub))
        print("N_CLASSES: "+ str(n_classes))
        
        X=np.load("input/sub_"+str(sub)+ "_compiled_RAW_downsampled_float32.npy")
        y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_sub/input/sub-"+str(sub)+"_compiled_rightSMA_scaled_per_sub_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels"]

        X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
        y=y.flatten()
        print("New shape for X: " + str(X.shape))
        print("New shape for y: "+str(y.shape))

        dir0= os.path.join(dir00, str(n_classes)+"_class")
#         os.mkdir(dir0)
        dir1=os.path.join(dir0, "comparison")
        dir2=os.path.join(dir0, "temporal_convolution")
        dir3=os.path.join(dir0, "spatial_convolution")
#         os.mkdir(dir1)
#         os.mkdir(dir2)
#         os.mkdir(dir3)

        ################################################################################################################################################################################################################################################################################

        n_folds = 5
        seed = 21
        shuffle_test = True
        EPOCHS=250

        kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
        count=0
        sommatoria=0

        for train_index, test_index in kfold.split(X):
            count=count+1

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            print("Train index for this split: "+ str(train_index)) 
            print("Number of samples for train set: "+str(train_index.shape[0]))
            print("Test index for this split: "+ str(test_index))
            print("Number of samples for test set: "+str(test_index.shape[0]))

            # Define the model architecture - 

            model=Sequential()

            ##################################################################

            model.add(Conv2D(8, (1, 64), padding = 'same',
                                           input_shape = (N_chan, N_samples_long, 1),
                                           use_bias = False, name="temporal_conv"))
            model.add(BatchNormalization(name="batchnorm_1"))
            model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                           depth_multiplier = 2,
                                           depthwise_constraint = max_norm(1.), name="spatial_conv"))
            model.add(BatchNormalization(name="batchnorm_2"))
            model.add(Activation('elu', name="activation_1"))
            model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
            model.add(Dropout(0.5, name="dropout_1"))

            model.add(SeparableConv2D(16, (1, 16),
                                           use_bias = False, padding = 'same', name="separable_conv"))
            model.add(BatchNormalization(name="batchnorm_3"))
            model.add(Activation('elu', name="activation_2"))
            model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
            model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

            model.add(Flatten(name = 'flatten'))

            model.add(Dense(numero_classi, name = 'dense', 
                                     kernel_constraint = max_norm(0.25)))
            model.add(Activation('softmax', name = 'softmax'))


            # Define the optimizer
            optimizer= optimizers.Adam(
            learning_rate= 1e-4,
            weight_decay= 0
            )
            model.compile(optimizer=optimizer,
                           loss=loss_fn,
                           metrics=['accuracy'])

            evaluation.append(model.fit(X_train, y_train, batch_size=16,
                      epochs=EPOCHS, 
                      validation_data=(X_test, y_test), 
                      verbose=0, workers=1)
                         )

            # Iteration = fold, i am just saving the model for that fold
            iteration.append(model)

            confusion_matrix_x_test.append(X_test)
            confusion_matrix_y_test.append(y_test)

            #Plotting confusion matrix
            pred=model.predict(X_test)
            y_test_pred= np.argmax(pred, axis=1)

            confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
            plt.figure()
            metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
            plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
            plt.close()

            validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

            PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

            #Salvo risultati di singolo fold
            sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
            sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


            # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
            var= (model.get_layer("temporal_conv").weights)
            for lallo in range(8):
                plt.figure()
                plt.title("temp_conv_"+str(lallo))
                plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
                plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
                temp= (var[0][0,:,0][:,lallo]).numpy()
                sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
                plt.close()


            var_2= (model.get_layer("spatial_conv").weights)
            reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
            for lallo in range(16):
                nump= reshaped_var_2[:,lallo].numpy()
                sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

        ###################################################################################################################

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
        min_temp_loss=10
        min_temp_acc=10
        max_temp_loss=0
        max_temp_acc=0

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['loss'])
            if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['val_loss'])
            if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['loss'])
            if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['val_loss'])

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['accuracy'])
            if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
            if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['accuracy'])
            if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['loss']
            loss_vec_test= evaluation[idx].history['val_loss']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title('Loss across epochs for fold: '+str(idx))
            plt.ylim([min_temp_loss, max_temp_loss])
            plt.legend()

            plt.savefig(dir1+"/loss_kfold_"+str(idx))
            plt.close()

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['accuracy']
            loss_vec_test= evaluation[idx].history['val_accuracy']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title('Accuracy across epochs for fold: '+str(idx))
            plt.ylim([min_temp_acc, max_temp_acc])
            plt.legend()

            plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
            plt.close()

        #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
        accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
        accuratezza[0]=(np.mean(validation_acc))
        accuratezza[1]=(statistics.pstdev(validation_acc))


        precisione=[]
        recall=[]
        f1_score=[]
        support=[]
        for classe in range(n_classes):
            precision_temp=[]
            recall_temp=[]
            f1_score_temp=[]
            support_temp=[]
            for idx in range(n_folds):
                precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
                recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
                f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
                support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

            precisione.append(np.mean(precision_temp))
            precisione.append(statistics.pstdev(precision_temp))
            recall.append(np.mean(recall_temp))
            recall.append(statistics.pstdev(recall_temp))
            f1_score.append(np.mean(f1_score_temp))
            f1_score.append(statistics.pstdev(f1_score_temp))    
            support.append(np.mean(support_temp))
            support.append(statistics.pstdev(support_temp)) 

        sommario=[]
        sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
        sio.savemat(dir0+"/sommario.mat", {"array": sommario})

        gianfranco=model.predict(X)
        gianfranco2=np.argmax(gianfranco, axis=1)
        sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



SUBJECT: 09
N_CLASSES: 50
New shape for X: (3048, 31, 250, 1)
New shape for y: (3048,)
Train index for this split: [   0    1    2 ... 3045 3046 3047]
Number of samples for train set: 2438
Test index for this split: [  10   12   16   19   21   23   29   35   36   39   44   54   58   65
   68   72   75   85   86   87   88   99  102  110  116  119  127  128
  133  136  142  146  155  169  172  174  187  190  200  201  212  215
  216  224  242  254  258  266  269  270  272  275  278  282  286  293
  294  295  297  304  305  309  311  316  319  328  333  335  347  350
  358  360  362  365  369  370  374  387  391  403  407  413  418  423
  430  441  450  452  457  458  465  480  490  500  507  511  527  537
  540  548  550  551  552  553  555  556  557  558  564  566  571  575
  579  581  584  590  592  595  596  602  606  611  621  623  624  632
  635  637  642  647  649  657  664  668  675  676  679  684  685  693
  697  701  705  708  710  712  713  717  718  719  721  724  730  734
  7

2023-04-03 06:04:19.020538: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 06:04:19.022354: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-03 06:04:19.022586: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-03 06:04:19.022730: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

20/20 [==============================] - 0s 2ms/step


/home/malfonzo/akulejo/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/malfonzo/akulejo/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/malfonzo/akulejo/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/malfonzo/akulejo/lib/python3.1

Train index for this split: [   0    1    3 ... 3043 3044 3047]
Number of samples for train set: 2438
Test index for this split: [   2    4   13   14   25   26   37   40   41   42   46   55   60   66
   77   81   89   91   94  103  104  109  112  118  125  137  140  147
  153  156  157  159  162  165  166  171  173  182  188  211  213  217
  221  222  229  237  250  259  261  265  267  271  273  276  277  279
  283  284  287  291  300  303  307  312  315  321  340  342  344  353
  368  371  373  375  379  393  394  395  396  406  410  439  443  451
  455  456  466  473  479  482  487  488  492  505  508  513  518  523
  534  535  538  546  547  561  565  568  570  582  597  605  615  616
  618  622  633  648  651  653  658  659  660  663  666  677  695  698
  704  706  709  714  715  726  727  731  732  743  745  746  750  755
  756  759  768  769  778  781  807  818  830  832  835  839  843  844
  848  851  857  866  871  876  877  882  885  893  897  898  904  910
  917  921  923  93

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [3]:
# PER SUB & PROTOCOL
SUBJECTS=[6,7,8,9,11,12,13,14,16,17]
for temp_sub in SUBJECTS:
    CLASSES= [2,3,5,10,50]
    #questa va applicata a for loop di subject che deve essere il più esterno 
    sub="{:02d}".format(temp_sub)

    dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_sub_protocol/output", "sub_"+str(sub))
    os.mkdir(dir00)
    for n_classes in CLASSES:
        #Define loss function
        loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
        LABELS= list(range(0,n_classes))
        
        if n_classes==50:
            numero_classi=5
        else:
            numero_classi=n_classes

        evaluation=[]
        iteration=[]
        confusion_matrix_x_test=[]
        confusion_matrix_y_test= []
        validation_acc=[]
        PERFORMANCE=[]

        print("SUBJECT: "+ str(sub))
        print("N_CLASSES: "+ str(n_classes))
        
        X=np.load("input/sub_"+str(sub)+ "_compiled_RAW_downsampled_float32.npy")
        y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_sub_protocol/input/sub-"+str(sub)+"_compiled_rightSMA_scaled_per_sub_protocol_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels"]

        X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
        y=y.flatten()
        print("New shape for X: " + str(X.shape))
        print("New shape for y: "+str(y.shape))

        dir0= os.path.join(dir00, str(n_classes)+"_class")
        os.mkdir(dir0)
        dir1=os.path.join(dir0, "comparison")
        dir2=os.path.join(dir0, "temporal_convolution")
        dir3=os.path.join(dir0, "spatial_convolution")
        os.mkdir(dir1)
        os.mkdir(dir2)
        os.mkdir(dir3)

        ################################################################################################################################################################################################################################################################################

        n_folds = 5
        seed = 24
        shuffle_test = True
        EPOCHS=250

        kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
        count=0
        sommatoria=0

        for train_index, test_index in kfold.split(X):
            count=count+1

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            print("Train index for this split: "+ str(train_index)) 
            print("Number of samples for train set: "+str(train_index.shape[0]))
            print("Test index for this split: "+ str(test_index))
            print("Number of samples for test set: "+str(test_index.shape[0]))

            # Define the model architecture - 

            model=Sequential()

            ##################################################################

            model.add(Conv2D(8, (1, 64), padding = 'same',
                                           input_shape = (N_chan, N_samples_long, 1),
                                           use_bias = False, name="temporal_conv"))
            model.add(BatchNormalization(name="batchnorm_1"))
            model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                           depth_multiplier = 2,
                                           depthwise_constraint = max_norm(1.), name="spatial_conv"))
            model.add(BatchNormalization(name="batchnorm_2"))
            model.add(Activation('elu', name="activation_1"))
            model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
            model.add(Dropout(0.5, name="dropout_1"))

            model.add(SeparableConv2D(16, (1, 16),
                                           use_bias = False, padding = 'same', name="separable_conv"))
            model.add(BatchNormalization(name="batchnorm_3"))
            model.add(Activation('elu', name="activation_2"))
            model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
            model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

            model.add(Flatten(name = 'flatten'))

            model.add(Dense(numero_classi, name = 'dense', 
                                     kernel_constraint = max_norm(0.25)))
            model.add(Activation('softmax', name = 'softmax'))


            # Define the optimizer
            optimizer= optimizers.Adam(
            learning_rate= 1e-4,
            weight_decay= 0
            )
            model.compile(optimizer=optimizer,
                           loss=loss_fn,
                           metrics=['accuracy'])

            evaluation.append(model.fit(X_train, y_train, batch_size=16,
                      epochs=EPOCHS, 
                      validation_data=(X_test, y_test), 
                      verbose=0, workers=1)
                         )

            # Iteration = fold, i am just saving the model for that fold
            iteration.append(model)

            confusion_matrix_x_test.append(X_test)
            confusion_matrix_y_test.append(y_test)

            #Plotting confusion matrix
            pred=model.predict(X_test)
            y_test_pred= np.argmax(pred, axis=1)

            confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
            plt.figure()
            metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
            plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
            plt.close()

            validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

            PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

            #Salvo risultati di singolo fold
            sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
            sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


            # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
            var= (model.get_layer("temporal_conv").weights)
            for lallo in range(8):
                plt.figure()
                plt.title("temp_conv_"+str(lallo))
                plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
                plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
                temp= (var[0][0,:,0][:,lallo]).numpy()
                sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
                plt.close()


            var_2= (model.get_layer("spatial_conv").weights)
            reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
            for lallo in range(16):
                nump= reshaped_var_2[:,lallo].numpy()
                sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

        ###################################################################################################################

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
        min_temp_loss=10
        min_temp_acc=10
        max_temp_loss=0
        max_temp_acc=0

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['loss'])
            if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['val_loss'])
            if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['loss'])
            if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['val_loss'])

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['accuracy'])
            if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
            if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['accuracy'])
            if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['loss']
            loss_vec_test= evaluation[idx].history['val_loss']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title('Loss across epochs for fold: '+str(idx))
            plt.ylim([min_temp_loss, max_temp_loss])
            plt.legend()

            plt.savefig(dir1+"/loss_kfold_"+str(idx))
            plt.close()

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['accuracy']
            loss_vec_test= evaluation[idx].history['val_accuracy']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title('Accuracy across epochs for fold: '+str(idx))
            plt.ylim([min_temp_acc, max_temp_acc])
            plt.legend()

            plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
            plt.close()

        #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
        accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
        accuratezza[0]=(np.mean(validation_acc))
        accuratezza[1]=(statistics.pstdev(validation_acc))


        precisione=[]
        recall=[]
        f1_score=[]
        support=[]
        for classe in range(n_classes):
            precision_temp=[]
            recall_temp=[]
            f1_score_temp=[]
            support_temp=[]
            for idx in range(n_folds):
                precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
                recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
                f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
                support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

            precisione.append(np.mean(precision_temp))
            precisione.append(statistics.pstdev(precision_temp))
            recall.append(np.mean(recall_temp))
            recall.append(statistics.pstdev(recall_temp))
            f1_score.append(np.mean(f1_score_temp))
            f1_score.append(statistics.pstdev(f1_score_temp))    
            support.append(np.mean(support_temp))
            support.append(statistics.pstdev(support_temp)) 

        sommario=[]
        sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
        sio.savemat(dir0+"/sommario.mat", {"array": sommario})

        gianfranco=model.predict(X)
        gianfranco2=np.argmax(gianfranco, axis=1)
        sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



SUBJECT: 06
N_CLASSES: 2
New shape for X: (3048, 31, 250, 1)
New shape for y: (3048,)
Train index for this split: [   0    1    2 ... 3045 3046 3047]
Number of samples for train set: 2438
Test index for this split: [   3    9   10   19   22   25   28   38   52   53   58   61   63   67
   72   77   89   90   92   96   99  100  104  109  116  117  118  126
  131  136  145  149  154  155  156  164  176  179  186  208  214  215
  228  231  237  241  246  260  265  267  278  282  285  287  292  295
  301  307  308  313  320  322  324  327  330  334  337  338  342  351
  355  367  368  382  385  388  389  400  401  406  408  416  424  438
  440  449  455  456  460  461  472  480  486  493  498  502  513  525
  527  537  538  542  555  565  569  581  587  589  591  592  599  606
  607  623  626  627  628  631  638  641  647  656  658  661  664  666
  670  679  687  688  691  692  694  699  727  733  734  738  743  744
  746  754  756  759  760  772  786  797  801  806  807  810  817  824
  82

2023-03-30 00:12:42.155216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 00:12:42.157061: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-30 00:12:42.157288: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-30 00:12:42.157446: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

20/20 [==============================] - 0s 2ms/step
Train index for this split: [   0    1    2 ... 3045 3046 3047]
Number of samples for train set: 2438
Test index for this split: [  16   21   23   27   31   35   40   42   46   51   54   55   57   68
   71   81   87   91   94   98  103  105  111  113  114  125  132  133
  143  153  159  161  162  169  185  188  189  197  203  210  216  217
  220  235  236  239  242  245  249  251  270  271  275  279  280  290
  293  296  300  306  310  314  315  317  332  333  345  346  348  354
  356  373  374  381  383  386  391  396  398  399  403  407  414  421
  429  433  437  442  462  468  469  470  471  487  496  497  503  517
  522  526  528  529  531  533  540  548  549  550  558  560  567  570
  574  580  584  585  588  595  601  602  615  616  619  630  633  637
  645  649  657  660  663  667  668  671  672  674  681  684  695  697
  701  706  708  710  713  715  716  717  752  761  764  771  779  781
  783  785  788  792  804  805  812  

2023-03-30 00:15:55.132074: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


20/20 [==============================] - 0s 2ms/step
Train index for this split: [   0    2    3 ... 3042 3045 3047]
Number of samples for train set: 2438
Test index for this split: [   1    6    7   13   14   15   20   34   37   41   47   50   59   60
   75   95   97  102  108  115  120  124  127  129  130  137  142  150
  152  172  174  178  181  194  200  204  221  223  234  244  248  250
  252  254  255  258  259  264  268  269  273  281  291  294  299  304
  305  326  329  344  349  350  352  362  363  366  370  378  380  392
  393  402  409  410  417  427  430  431  434  444  447  451  458  459
  464  465  473  476  477  478  489  490  500  501  508  509  511  515
  519  532  543  544  547  552  557  559  561  562  563  573  579  582
  583  596  598  600  604  609  617  620  624  634  636  640  642  643
  644  648  652  655  662  673  677  682  683  689  690  709  712  714
  718  732  735  736  737  739  742  747  755  757  763  768  773  777
  778  794  796  798  811  819  821  

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [5]:
# PER RUN 
# SUBJECTS=[3,4,5,6,7,8,9,11,12,13,14,16,17]
SUBJECTS=[17]
for temp_sub in SUBJECTS:
#     CLASSES= [2,3,5,10,50]
    CLASSES= [3,5,10,50]
    #questa va applicata a for loop di subject che deve essere il più esterno 
    sub="{:02d}".format(temp_sub)

    dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_run/output", "sub_"+str(sub))
#     os.mkdir(dir00)
    for n_classes in CLASSES:
        #Define loss function
        loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
        LABELS= list(range(0,n_classes))
        
        if n_classes==50:
            numero_classi=5
        else:
            numero_classi=n_classes

        evaluation=[]
        iteration=[]
        confusion_matrix_x_test=[]
        confusion_matrix_y_test= []
        validation_acc=[]
        PERFORMANCE=[]

        print("SUBJECT: "+ str(sub))
        print("N_CLASSES: "+ str(n_classes))
        
        X=np.load("input/sub_"+str(sub)+ "_compiled_RAW_downsampled_float32.npy")
        y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_run/input/sub-"+str(sub)+"_compiled_rightSMA_scaled_per_run_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels"]

        X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
        y=y.flatten()
        print("New shape for X: " + str(X.shape))
        print("New shape for y: "+str(y.shape))

        dir0= os.path.join(dir00, str(n_classes)+"_class")
        os.mkdir(dir0)
        dir1=os.path.join(dir0, "comparison")
        dir2=os.path.join(dir0, "temporal_convolution")
        dir3=os.path.join(dir0, "spatial_convolution")
        os.mkdir(dir1)
        os.mkdir(dir2)
        os.mkdir(dir3)

        ################################################################################################################################################################################################################################################################################

        n_folds = 5
#         seed = 21
        shuffle_test = True
        EPOCHS=250

        kfold = KFold(n_splits = n_folds, shuffle = shuffle_test)
        count=0
        sommatoria=0

        for train_index, test_index in kfold.split(X):
            count=count+1

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            print("Train index for this split: "+ str(train_index)) 
            print("Number of samples for train set: "+str(train_index.shape[0]))
            print("Test index for this split: "+ str(test_index))
            print("Number of samples for test set: "+str(test_index.shape[0]))

            # Define the model architecture - 

            model=Sequential()

            ##################################################################

            model.add(Conv2D(8, (1, 64), padding = 'same',
                                           input_shape = (N_chan, N_samples_long, 1),
                                           use_bias = False, name="temporal_conv"))
            model.add(BatchNormalization(name="batchnorm_1"))
            model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                           depth_multiplier = 2,
                                           depthwise_constraint = max_norm(1.), name="spatial_conv"))
            model.add(BatchNormalization(name="batchnorm_2"))
            model.add(Activation('elu', name="activation_1"))
            model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
            model.add(Dropout(0.5, name="dropout_1"))

            model.add(SeparableConv2D(16, (1, 16),
                                           use_bias = False, padding = 'same', name="separable_conv"))
            model.add(BatchNormalization(name="batchnorm_3"))
            model.add(Activation('elu', name="activation_2"))
            model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
            model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

            model.add(Flatten(name = 'flatten'))

            model.add(Dense(numero_classi, name = 'dense', 
                                     kernel_constraint = max_norm(0.25)))
            model.add(Activation('softmax', name = 'softmax'))


            # Define the optimizer
            optimizer= optimizers.Adam(
            learning_rate= 1e-4,
            weight_decay= 0
            )
            model.compile(optimizer=optimizer,
                           loss=loss_fn,
                           metrics=['accuracy'])

            evaluation.append(model.fit(X_train, y_train, batch_size=16,
                      epochs=EPOCHS, 
                      validation_data=(X_test, y_test), 
                      verbose=0, workers=1)
                         )

            # Iteration = fold, i am just saving the model for that fold
            iteration.append(model)

            confusion_matrix_x_test.append(X_test)
            confusion_matrix_y_test.append(y_test)

            #Plotting confusion matrix
            pred=model.predict(X_test)
            y_test_pred= np.argmax(pred, axis=1)

            confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
            plt.figure()
            metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
            plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
            plt.close()

            validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

            PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

            #Salvo risultati di singolo fold
            sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
            sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


            # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
            var= (model.get_layer("temporal_conv").weights)
            for lallo in range(8):
                plt.figure()
                plt.title("temp_conv_"+str(lallo))
                plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
                plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
                temp= (var[0][0,:,0][:,lallo]).numpy()
                sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
                plt.close()


            var_2= (model.get_layer("spatial_conv").weights)
            reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
            for lallo in range(16):
                nump= reshaped_var_2[:,lallo].numpy()
                sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

        ###################################################################################################################

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
        min_temp_loss=10
        min_temp_acc=10
        max_temp_loss=0
        max_temp_acc=0

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['loss'])
            if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
                min_temp_loss=np.min(evaluation[idx].history['val_loss'])
            if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['loss'])
            if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
                max_temp_loss=np.max(evaluation[idx].history['val_loss'])

        for idx in range(n_folds):
            if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['accuracy'])
            if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
                min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
            if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['accuracy'])
            if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
                max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['loss']
            loss_vec_test= evaluation[idx].history['val_loss']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title('Loss across epochs for fold: '+str(idx))
            plt.ylim([min_temp_loss, max_temp_loss])
            plt.legend()

            plt.savefig(dir1+"/loss_kfold_"+str(idx))
            plt.close()

        #plot accuracy and loss function across epochs
        epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

        for idx in range(n_folds):
            loss_vec_train= evaluation[idx].history['accuracy']
            loss_vec_test= evaluation[idx].history['val_accuracy']

            plt.figure()
            plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
            plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title('Accuracy across epochs for fold: '+str(idx))
            plt.ylim([min_temp_acc, max_temp_acc])
            plt.legend()

            plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
            plt.close()

        #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
        accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
        accuratezza[0]=(np.mean(validation_acc))
        accuratezza[1]=(statistics.pstdev(validation_acc))


        precisione=[]
        recall=[]
        f1_score=[]
        support=[]
        for classe in range(numero_classi):
            precision_temp=[]
            recall_temp=[]
            f1_score_temp=[]
            support_temp=[]
            for idx in range(n_folds):
                precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
                recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
                f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
                support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

            precisione.append(np.mean(precision_temp))
            precisione.append(statistics.pstdev(precision_temp))
            recall.append(np.mean(recall_temp))
            recall.append(statistics.pstdev(recall_temp))
            f1_score.append(np.mean(f1_score_temp))
            f1_score.append(statistics.pstdev(f1_score_temp))    
            support.append(np.mean(support_temp))
            support.append(statistics.pstdev(support_temp)) 

        sommario=[]
        sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
        sio.savemat(dir0+"/sommario.mat", {"array": sommario})

        gianfranco=model.predict(X)
        gianfranco2=np.argmax(gianfranco, axis=1)
        sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



SUBJECT: 17
N_CLASSES: 3
New shape for X: (3048, 31, 250, 1)
New shape for y: (3048,)
Train index for this split: [   2    3    4 ... 3043 3046 3047]
Number of samples for train set: 2438
Test index for this split: [   0    1    6    7   10   14   17   19   23   34   36   37   39   41
   46   55   64   69   76   83   88   95  105  114  124  126  128  129
  134  136  139  145  147  148  152  163  171  173  179  180  181  187
  193  197  206  220  226  233  239  248  252  256  258  259  263  266
  273  283  285  286  287  303  307  309  315  316  319  322  327  336
  339  341  343  348  349  353  354  356  387  394  395  401  404  414
  416  421  429  433  434  454  463  485  494  495  504  506  508  512
  520  523  528  531  534  536  538  547  559  566  568  592  598  611
  613  620  623  624  629  637  644  645  646  649  652  653  656  659
  663  671  677  682  683  685  687  689  690  691  707  720  722  726
  727  735  736  744  750  754  755  765  769  770  771  789  798  807
  81

2023-04-03 06:20:54.952973: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_5/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-04-03 06:20:55.192496: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.2.89, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-03 06:20:55.297103: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.2.89, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often suffic

20/20 [==============================] - 0s 2ms/step
Train index for this split: [   0    1    2 ... 3045 3046 3047]
Number of samples for train set: 2438
Test index for this split: [   9   11   12   13   24   27   28   29   31   40   43   51   53   54
   57   62   67   68   72   73   75   77   78   84   89   90   96  102
  112  119  120  123  133  140  143  149  150  151  155  156  165  188
  191  195  204  207  209  212  213  215  217  224  225  231  232  234
  235  236  238  242  243  246  247  250  254  260  261  262  268  269
  270  274  277  282  293  298  320  321  323  332  338  342  345  346
  357  362  363  364  366  368  375  382  385  390  405  409  410  411
  420  424  431  440  455  461  466  467  472  474  484  487  493  500
  501  510  514  515  521  525  527  529  539  549  556  561  563  564
  565  570  574  579  580  581  585  586  587  589  595  596  599  605
  609  610  614  616  627  628  632  643  655  661  664  668  670  672
  679  688  694  708  711  713  714  

2023-04-03 06:24:02.325705: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_6/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


20/20 [==============================] - 0s 2ms/step


...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

# ALL-per SUB

In [ ]:
# PER SUB 
# ########SUBJECTS=[1,2,3,4,5,6,7,8,9,11,12,13,14,16,17]
# SUBJECTS=[11,12,13,14,16,17]
# for temp_sub in SUBJECTS:
CLASSES= [2,3,5,10,50]
#questa va applicata a for loop di subject che deve essere il più esterno 
#     sub="{:02d}".format(temp_sub)

dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_sub/output", "all")
os.mkdir(dir00)
for n_classes in CLASSES:
    #Define loss function
    loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    LABELS= list(range(0,n_classes))

    if n_classes==50:
        numero_classi=5
    else:
        numero_classi=n_classes

    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    PERFORMANCE=[]

    X=np.load("input/RAW_regression_all_subs_float32.npy")
    y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_sub/input/Compiled_rightSMA_all_scaled_per_sub_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels_all"]

    X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    y=y.flatten()
    print("Shape for X: " + str(X.shape))
    print("Shape for y: "+str(y.shape))

    X= X[762:,:,:,:]
    y= y[762:]
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))
    
    dir0= os.path.join(dir00, str(n_classes)+"_class")
    os.mkdir(dir0)
    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
    os.mkdir(dir1)
    os.mkdir(dir2)
    os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################

    n_folds = 5
    seed = 21
    shuffle_test = True
    EPOCHS=250
    DROPOUT_TEST=0.25

    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 1),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(DROPOUT_TEST, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(numero_classi, name = 'dense', 
                                 kernel_constraint = max_norm(0.25)))
        model.add(Activation('softmax', name = 'softmax'))


        # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= 1e-4,
        weight_decay= 0
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=['accuracy'])

        evaluation.append(model.fit(X_train, y_train,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=2, workers=1)
                     )

        # Iteration = fold, i am just saving the model for that fold
        iteration.append(model)

        confusion_matrix_x_test.append(X_test)
        confusion_matrix_y_test.append(y_test)

        #Plotting confusion matrix
        pred=model.predict(X_test)
        y_test_pred= np.argmax(pred, axis=1)

        confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
        plt.figure()
        metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
        plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
        plt.close()

        validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

        PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

        #Salvo risultati di singolo fold
        sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
        sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


        # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
            plt.close()


        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

    ###################################################################################################################

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['accuracy'])
        if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
        if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['accuracy'])
        if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for fold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['accuracy']
        loss_vec_test= evaluation[idx].history['val_accuracy']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Accuracy across epochs for fold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
        plt.close()

    #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
    accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
    accuratezza[0]=(np.mean(validation_acc))
    accuratezza[1]=(statistics.pstdev(validation_acc))


    precisione=[]
    recall=[]
    f1_score=[]
    support=[]
    for classe in range(numero_classi):
        precision_temp=[]
        recall_temp=[]
        f1_score_temp=[]
        support_temp=[]
        for idx in range(n_folds):
            precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
            recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
            f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
            support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

        precisione.append(np.mean(precision_temp))
        precisione.append(statistics.pstdev(precision_temp))
        recall.append(np.mean(recall_temp))
        recall.append(statistics.pstdev(recall_temp))
        f1_score.append(np.mean(f1_score_temp))
        f1_score.append(statistics.pstdev(f1_score_temp))    
        support.append(np.mean(support_temp))
        support.append(statistics.pstdev(support_temp)) 

        sio.savemat(dir0+"/precisione.mat", {"array":precisione})
        sio.savemat(dir0+"/recall.mat", {"array":recall})
        sio.savemat(dir0+"/f1_score.mat", {"array":f1_score})
        sio.savemat(dir0+"/support.mat", {"array":support})

#     sommario=[]
#     sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
#     sio.savemat(dir0+"/sommario.mat", {"array": sommario})

    gianfranco=model.predict(X)
    gianfranco2=np.argmax(gianfranco, axis=1)
    sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



Shape for X: (45720, 31, 250, 1)
Shape for y: (45720,)
New shape for X: (44958, 31, 250, 1)
New shape for y: (44958,)
Train index for this split: [    0     1     3 ... 44953 44955 44957]
Number of samples for train set: 35966
Test index for this split: [    2     7    21 ... 44951 44954 44956]
Number of samples for test set: 8992
Epoch 1/250


2023-04-01 03:34:38.342081: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_308/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1124/1124 - 8s - loss: 0.5759 - accuracy: 0.7143 - val_loss: 0.5178 - val_accuracy: 0.7459 - 8s/epoch - 7ms/step
Epoch 2/250
1124/1124 - 6s - loss: 0.4691 - accuracy: 0.7805 - val_loss: 0.4545 - val_accuracy: 0.8062 - 6s/epoch - 5ms/step
Epoch 3/250
1124/1124 - 6s - loss: 0.4272 - accuracy: 0.8087 - val_loss: 0.4206 - val_accuracy: 0.8238 - 6s/epoch - 5ms/step
Epoch 4/250
1124/1124 - 6s - loss: 0.4069 - accuracy: 0.8204 - val_loss: 0.4011 - val_accuracy: 0.8297 - 6s/epoch - 5ms/step
Epoch 5/250
1124/1124 - 5s - loss: 0.3962 - accuracy: 0.8244 - val_loss: 0.3952 - val_accuracy: 0.8321 - 5s/epoch - 5ms/step
Epoch 6/250
1124/1124 - 6s - loss: 0.3920 - accuracy: 0.8281 - val_loss: 0.3846 - val_accuracy: 0.8349 - 6s/epoch - 5ms/step
Epoch 7/250
1124/1124 - 6s - loss: 0.3843 - accuracy: 0.8315 - val_loss: 0.3823 - val_accuracy: 0.8375 - 6s/epoch - 5ms/step
Epoch 8/250
1124/1124 - 5s - loss: 0.3809 - accuracy: 0.8301 - val_loss: 0.3809 - val_accuracy: 0.8359 - 5s/epoch - 5ms/step
Epoch 9/250


Epoch 67/250
1124/1124 - 6s - loss: 0.3424 - accuracy: 0.8426 - val_loss: 0.3422 - val_accuracy: 0.8387 - 6s/epoch - 5ms/step
Epoch 68/250
1124/1124 - 5s - loss: 0.3416 - accuracy: 0.8447 - val_loss: 0.3411 - val_accuracy: 0.8416 - 5s/epoch - 5ms/step
Epoch 69/250
1124/1124 - 5s - loss: 0.3416 - accuracy: 0.8446 - val_loss: 0.3408 - val_accuracy: 0.8413 - 5s/epoch - 5ms/step
Epoch 70/250
1124/1124 - 6s - loss: 0.3413 - accuracy: 0.8438 - val_loss: 0.3417 - val_accuracy: 0.8401 - 6s/epoch - 5ms/step
Epoch 71/250
1124/1124 - 6s - loss: 0.3413 - accuracy: 0.8441 - val_loss: 0.3394 - val_accuracy: 0.8409 - 6s/epoch - 5ms/step
Epoch 72/250
1124/1124 - 6s - loss: 0.3412 - accuracy: 0.8427 - val_loss: 0.3429 - val_accuracy: 0.8407 - 6s/epoch - 5ms/step
Epoch 73/250
1124/1124 - 5s - loss: 0.3426 - accuracy: 0.8420 - val_loss: 0.3431 - val_accuracy: 0.8409 - 5s/epoch - 5ms/step
Epoch 74/250
1124/1124 - 5s - loss: 0.3396 - accuracy: 0.8445 - val_loss: 0.3439 - val_accuracy: 0.8381 - 5s/epoch - 5

Epoch 132/250
1124/1124 - 5s - loss: 0.3327 - accuracy: 0.8459 - val_loss: 0.3377 - val_accuracy: 0.8396 - 5s/epoch - 5ms/step
Epoch 133/250
1124/1124 - 5s - loss: 0.3341 - accuracy: 0.8460 - val_loss: 0.3363 - val_accuracy: 0.8394 - 5s/epoch - 5ms/step
Epoch 134/250
1124/1124 - 6s - loss: 0.3321 - accuracy: 0.8475 - val_loss: 0.3392 - val_accuracy: 0.8385 - 6s/epoch - 5ms/step
Epoch 135/250
1124/1124 - 5s - loss: 0.3321 - accuracy: 0.8474 - val_loss: 0.3349 - val_accuracy: 0.8412 - 5s/epoch - 5ms/step
Epoch 136/250
1124/1124 - 5s - loss: 0.3324 - accuracy: 0.8464 - val_loss: 0.3341 - val_accuracy: 0.8417 - 5s/epoch - 5ms/step
Epoch 137/250
1124/1124 - 6s - loss: 0.3330 - accuracy: 0.8460 - val_loss: 0.3337 - val_accuracy: 0.8405 - 6s/epoch - 5ms/step
Epoch 138/250
1124/1124 - 6s - loss: 0.3327 - accuracy: 0.8482 - val_loss: 0.3400 - val_accuracy: 0.8376 - 6s/epoch - 5ms/step
Epoch 139/250
1124/1124 - 6s - loss: 0.3347 - accuracy: 0.8467 - val_loss: 0.3375 - val_accuracy: 0.8400 - 6s/e

...

2023-04-01 04:20:57.430377: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_310/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1124/1124 - 8s - loss: 0.5820 - accuracy: 0.7044 - val_loss: 0.5235 - val_accuracy: 0.7484 - 8s/epoch - 7ms/step
Epoch 2/250
1124/1124 - 5s - loss: 0.4761 - accuracy: 0.7766 - val_loss: 0.4462 - val_accuracy: 0.8141 - 5s/epoch - 5ms/step
Epoch 3/250
1124/1124 - 6s - loss: 0.4323 - accuracy: 0.8018 - val_loss: 0.4161 - val_accuracy: 0.8265 - 6s/epoch - 5ms/step
Epoch 4/250
1124/1124 - 5s - loss: 0.4109 - accuracy: 0.8123 - val_loss: 0.3936 - val_accuracy: 0.8361 - 5s/epoch - 5ms/step
Epoch 5/250
1124/1124 - 5s - loss: 0.4005 - accuracy: 0.8188 - val_loss: 0.3851 - val_accuracy: 0.8415 - 5s/epoch - 5ms/step
Epoch 6/250
1124/1124 - 5s - loss: 0.3924 - accuracy: 0.8222 - val_loss: 0.3756 - val_accuracy: 0.8421 - 5s/epoch - 5ms/step
Epoch 7/250
1124/1124 - 5s - loss: 0.3862 - accuracy: 0.8244 - val_loss: 0.3744 - val_accuracy: 0.8412 - 5s/epoch - 5ms/step
Epoch 8/250
1124/1124 - 5s - loss: 0.3836 - accuracy: 0.8276 - val_loss: 0.3671 - val_accuracy: 0.8448 - 5s/epoch - 5ms/step
Epoch 9/250


Epoch 67/250
1124/1124 - 5s - loss: 0.3460 - accuracy: 0.8413 - val_loss: 0.3395 - val_accuracy: 0.8481 - 5s/epoch - 5ms/step
Epoch 68/250
1124/1124 - 6s - loss: 0.3448 - accuracy: 0.8419 - val_loss: 0.3370 - val_accuracy: 0.8464 - 6s/epoch - 5ms/step
Epoch 69/250
1124/1124 - 5s - loss: 0.3462 - accuracy: 0.8410 - val_loss: 0.3392 - val_accuracy: 0.8450 - 5s/epoch - 5ms/step
Epoch 70/250
1124/1124 - 5s - loss: 0.3458 - accuracy: 0.8416 - val_loss: 0.3382 - val_accuracy: 0.8466 - 5s/epoch - 5ms/step
Epoch 71/250
1124/1124 - 5s - loss: 0.3448 - accuracy: 0.8403 - val_loss: 0.3384 - val_accuracy: 0.8469 - 5s/epoch - 5ms/step
Epoch 72/250
1124/1124 - 5s - loss: 0.3461 - accuracy: 0.8409 - val_loss: 0.3358 - val_accuracy: 0.8472 - 5s/epoch - 5ms/step
Epoch 73/250
1124/1124 - 5s - loss: 0.3451 - accuracy: 0.8410 - val_loss: 0.3364 - val_accuracy: 0.8468 - 5s/epoch - 5ms/step
Epoch 74/250
1124/1124 - 5s - loss: 0.3453 - accuracy: 0.8400 - val_loss: 0.3375 - val_accuracy: 0.8473 - 5s/epoch - 5

Epoch 132/250
1124/1124 - 5s - loss: 0.3352 - accuracy: 0.8439 - val_loss: 0.3293 - val_accuracy: 0.8461 - 5s/epoch - 5ms/step
Epoch 133/250
1124/1124 - 5s - loss: 0.3347 - accuracy: 0.8449 - val_loss: 0.3297 - val_accuracy: 0.8473 - 5s/epoch - 5ms/step
Epoch 134/250
1124/1124 - 6s - loss: 0.3382 - accuracy: 0.8421 - val_loss: 0.3315 - val_accuracy: 0.8461 - 6s/epoch - 5ms/step
Epoch 135/250
1124/1124 - 5s - loss: 0.3354 - accuracy: 0.8453 - val_loss: 0.3323 - val_accuracy: 0.8455 - 5s/epoch - 5ms/step
Epoch 136/250
1124/1124 - 5s - loss: 0.3369 - accuracy: 0.8438 - val_loss: 0.3293 - val_accuracy: 0.8465 - 5s/epoch - 5ms/step
Epoch 137/250
1124/1124 - 5s - loss: 0.3365 - accuracy: 0.8449 - val_loss: 0.3328 - val_accuracy: 0.8464 - 5s/epoch - 5ms/step
Epoch 138/250
1124/1124 - 5s - loss: 0.3349 - accuracy: 0.8449 - val_loss: 0.3310 - val_accuracy: 0.8464 - 5s/epoch - 5ms/step
Epoch 139/250
1124/1124 - 6s - loss: 0.3362 - accuracy: 0.8450 - val_loss: 0.3292 - val_accuracy: 0.8488 - 6s/e

Epoch 197/250
1124/1124 - 5s - loss: 0.3316 - accuracy: 0.8452 - val_loss: 0.3259 - val_accuracy: 0.8484 - 5s/epoch - 5ms/step
Epoch 198/250
1124/1124 - 6s - loss: 0.3316 - accuracy: 0.8461 - val_loss: 0.3285 - val_accuracy: 0.8464 - 6s/epoch - 5ms/step
Epoch 199/250
1124/1124 - 5s - loss: 0.3329 - accuracy: 0.8467 - val_loss: 0.3332 - val_accuracy: 0.8455 - 5s/epoch - 5ms/step
Epoch 200/250
1124/1124 - 5s - loss: 0.3332 - accuracy: 0.8443 - val_loss: 0.3288 - val_accuracy: 0.8469 - 5s/epoch - 5ms/step
Epoch 201/250
1124/1124 - 6s - loss: 0.3300 - accuracy: 0.8452 - val_loss: 0.3281 - val_accuracy: 0.8464 - 6s/epoch - 5ms/step
Epoch 202/250
1124/1124 - 6s - loss: 0.3302 - accuracy: 0.8453 - val_loss: 0.3281 - val_accuracy: 0.8486 - 6s/epoch - 5ms/step
Epoch 203/250
1124/1124 - 6s - loss: 0.3318 - accuracy: 0.8444 - val_loss: 0.3311 - val_accuracy: 0.8464 - 6s/epoch - 5ms/step
Epoch 204/250
1124/1124 - 5s - loss: 0.3303 - accuracy: 0.8458 - val_loss: 0.3285 - val_accuracy: 0.8473 - 5s/e

In [ ]:
# PER SUB & PROTOCOL
# SUBJECTS=[6,7,8,9,11,12,13,14,16,17]
# for temp_sub in SUBJECTS:
CLASSES= [2,3,5,10,50]
#questa va applicata a for loop di subject che deve essere il più esterno 
#     sub="{:02d}".format(temp_sub)

dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_sub_protocol/output", "all")
os.mkdir(dir00)
for n_classes in CLASSES:
    #Define loss function
    loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    LABELS= list(range(0,n_classes))

    if n_classes==50:
        numero_classi=5
    else:
        numero_classi=n_classes

    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    PERFORMANCE=[]

#     print("SUBJECT: "+ str(sub))
    print("N_CLASSES: "+ str(n_classes))

    X=np.load("input/RAW_regression_all_subs_float32.npy")
    y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_sub_protocol/input/Compiled_rightSMA_all_scaled_per_sub_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels_all"]

    X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    y=y.flatten()
    print("Shape for X: " + str(X.shape))
    print("Shape for y: "+str(y.shape))

    X= X[762:,:,:,:]
    y= y[762:]
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))

    dir0= os.path.join(dir00, str(n_classes)+"_class")
    os.mkdir(dir0)
    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
    os.mkdir(dir1)
    os.mkdir(dir2)
    os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################

    n_folds = 5
    seed = 24
    shuffle_test = True
    EPOCHS=250
    DROPOUT_TEST=0.25
    
    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 1),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(DROPOUT_TEST, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(numero_classi, name = 'dense', 
                                 kernel_constraint = max_norm(0.25)))
        model.add(Activation('softmax', name = 'softmax'))


        # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= 1e-4,
        weight_decay= 0
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=['accuracy'])

        evaluation.append(model.fit(X_train, y_train,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=0, workers=1)
                     )

        # Iteration = fold, i am just saving the model for that fold
        iteration.append(model)

        confusion_matrix_x_test.append(X_test)
        confusion_matrix_y_test.append(y_test)

        #Plotting confusion matrix
        pred=model.predict(X_test)
        y_test_pred= np.argmax(pred, axis=1)

        confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
        plt.figure()
        metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
        plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
        plt.close()

        validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

        PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

        #Salvo risultati di singolo fold
        sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
        sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


        # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
            plt.close()


        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

    ###################################################################################################################

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['accuracy'])
        if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
        if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['accuracy'])
        if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for fold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['accuracy']
        loss_vec_test= evaluation[idx].history['val_accuracy']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Accuracy across epochs for fold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
        plt.close()

    #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
    accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
    accuratezza[0]=(np.mean(validation_acc))
    accuratezza[1]=(statistics.pstdev(validation_acc))


    precisione=[]
    recall=[]
    f1_score=[]
    support=[]
    for classe in range(numero_classi):
        precision_temp=[]
        recall_temp=[]
        f1_score_temp=[]
        support_temp=[]
        for idx in range(n_folds):
            precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
            recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
            f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
            support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

        precisione.append(np.mean(precision_temp))
        precisione.append(statistics.pstdev(precision_temp))
        recall.append(np.mean(recall_temp))
        recall.append(statistics.pstdev(recall_temp))
        f1_score.append(np.mean(f1_score_temp))
        f1_score.append(statistics.pstdev(f1_score_temp))    
        support.append(np.mean(support_temp))
        support.append(statistics.pstdev(support_temp)) 

        sio.savemat(dir0+"/precisione.mat", {"array":precisione})
        sio.savemat(dir0+"/recall.mat", {"array":recall})
        sio.savemat(dir0+"/f1_score.mat", {"array":f1_score})
        sio.savemat(dir0+"/support.mat", {"array":support})

#     sommario=[]
#     sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
#     sio.savemat(dir0+"/sommario.mat", {"array": sommario})

    gianfranco=model.predict(X)
    gianfranco2=np.argmax(gianfranco, axis=1)
    sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})



In [ ]:
# PER RUN 
# SUBJECTS=[3,4,5,6,7,8,9,11,12,13,14,16,17]
# for temp_sub in SUBJECTS:
CLASSES= [2,3,5,10,50]
#questa va applicata a for loop di subject che deve essere il più esterno 
# sub="{:02d}".format(temp_sub)

dir00= os.path.join("FINAL_fuzzy_regressor/MinMax_per_run/output", "all")
os.mkdir(dir00)
for n_classes in CLASSES:
    #Define loss function
    loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    LABELS= list(range(0,n_classes))

    if n_classes==50:
        numero_classi=5
    else:
        numero_classi=n_classes

    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    PERFORMANCE=[]

    X=np.load("input/RAW_regression_all_subs_float32.npy")
    y= sio.loadmat("FINAL_fuzzy_regressor/MinMax_per_run/input/Compiled_rightSMA_all_scaled_per_sub_"+str(n_classes)+"class_balanced.mat")["compiled_rightSMA_labels_all"]

    X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    y=y.flatten()
    print("Shape for X: " + str(X.shape))
    print("Shape for y: "+str(y.shape))

    X= X[762:,:,:,:]
    y= y[762:]
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))

    dir0= os.path.join(dir00, str(n_classes)+"_class")
    os.mkdir(dir0)
    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
    os.mkdir(dir1)
    os.mkdir(dir2)
    os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################

    n_folds = 5
#         seed = 21
    shuffle_test = True
    EPOCHS=250
    DROPOUT_TEST=0.25

    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 1),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(DROPOUT_TEST, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(numero_classi, name = 'dense', 
                                 kernel_constraint = max_norm(0.25)))
        model.add(Activation('softmax', name = 'softmax'))


        # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= 1e-4,
        weight_decay= 0
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=['accuracy'])

        evaluation.append(model.fit(X_train, y_train,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=0, workers=1)
                     )

        # Iteration = fold, i am just saving the model for that fold
        iteration.append(model)

        confusion_matrix_x_test.append(X_test)
        confusion_matrix_y_test.append(y_test)

        #Plotting confusion matrix
        pred=model.predict(X_test)
        y_test_pred= np.argmax(pred, axis=1)

        confusion_matrix= metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
        plt.figure()
        metrics.ConfusionMatrixDisplay(confusion_matrix).plot()
        plt.savefig(dir1+"/confusion_matrix_kfold_"+str(count))
        plt.close()

        validation_acc.append(np.sum(y_test==y_test_pred)/y_test.shape[0])

        PERFORMANCE.append(classification_report(y_test, y_test_pred, labels=LABELS, output_dict=True))

        #Salvo risultati di singolo fold
        sio.savemat(dir1+"/y_pred_test_kfold"+str(count), {"array": y_test_pred})
        sio.savemat(dir1+"/y_test_kfold"+str(count), {"array": y_test})


        # PLOTTO FILTRI TEMPORALI E SPAZIALI E LI SALVO
        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo))
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/temp_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": temp})
            plt.close()


        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/spat_conv_kfold_"+str(count)+"_filter_"+str(lallo), {"array": nump})

    ###################################################################################################################

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['accuracy'])
        if (np.min(evaluation[idx].history['val_accuracy'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_accuracy'])
        if (np.max(evaluation[idx].history['accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['accuracy'])
        if (np.max(evaluation[idx].history['val_accuracy'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_accuracy'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for fold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['accuracy']
        loss_vec_test= evaluation[idx].history['val_accuracy']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Accuracy across epochs for fold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/accuracy_kfold_"+str(idx))
        plt.close()

    #SALVO VARIABILI STATISTICHE E MODELLO IN MODO DA PLOTTARLO

#         acc_temp=[]
    accuratezza=np.zeros(n_classes*2)
#         for idx in range(n_folds):
#             acc_temp.append(PERFORMANCE[idx]["accuracy"])
    accuratezza[0]=(np.mean(validation_acc))
    accuratezza[1]=(statistics.pstdev(validation_acc))


    precisione=[]
    recall=[]
    f1_score=[]
    support=[]
    for classe in range(numero_classi):
        precision_temp=[]
        recall_temp=[]
        f1_score_temp=[]
        support_temp=[]
        for idx in range(n_folds):
            precision_temp.append(PERFORMANCE[idx][str(classe)]["precision"])
            recall_temp.append(PERFORMANCE[idx][str(classe)]["recall"])
            f1_score_temp.append(PERFORMANCE[idx][str(classe)]["f1-score"])
            support_temp.append(PERFORMANCE[idx][str(classe)]["support"])

        precisione.append(np.mean(precision_temp))
        precisione.append(statistics.pstdev(precision_temp))
        recall.append(np.mean(recall_temp))
        recall.append(statistics.pstdev(recall_temp))
        f1_score.append(np.mean(f1_score_temp))
        f1_score.append(statistics.pstdev(f1_score_temp))    
        support.append(np.mean(support_temp))
        support.append(statistics.pstdev(support_temp)) 
        
        sio.savemat(dir0+"/precisione.mat", {"array":precisione})
        sio.savemat(dir0+"/recall.mat", {"array":recall})
        sio.savemat(dir0+"/f1_score.mat", {"array":f1_score})
        sio.savemat(dir0+"/support.mat", {"array":support})

#     sommario=[]
#     sommario=np.vstack((accuratezza,precisione,recall,f1_score,recall))
#     sio.savemat(dir0+"/sommario.mat", {"array": sommario})

    gianfranco=model.predict(X)
    gianfranco2=np.argmax(gianfranco, axis=1)
    sio.savemat(dir0+"/predizione_totale.mat", {"array": gianfranco2})

